In [1]:
from pyspark import SparkContext

In [2]:
import pyspark.sql.functions as f
from pyspark.sql.types import *
from pyspark.sql.functions import *

In [3]:
saf=sqlContext.read.csv("./data/0.csv",header=True)

In [4]:
saf.printSchema()

root
 |-- geoid: string (nullable = true)
 |-- name: string (nullable = true)
 |-- year: string (nullable = true)
 |-- total_est: string (nullable = true)
 |-- total_se: string (nullable = true)



In [5]:
sqlContext.read.csv("./data/0.csv",header=True).registerTempTable("zero")

/sw/pkgs/arc/spark/spark-3.2.1-bin-hadoop3.2/python/pyspark/sql/dataframe.py:138: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn(


In [6]:
sqlContext.read.csv("./data/1.csv",header=True).registerTempTable('One')

In [7]:
sqlContext.read.csv("./data/2.csv",header=True).registerTempTable('Two')
sqlContext.read.csv("./data/3.csv",header=True).registerTempTable('Three')
sqlContext.read.csv("./data/4.csv",header=True).registerTempTable('Four')

In [8]:
# Deal with the txt file of the birth rate
in3=sqlContext.read.csv("./data/birthrate.txt",header=True,sep='\t')

In [9]:
in3.columns

['Notes',
 'Year',
 'YearCode',
 'State',
 'StateCode',
 'Births',
 'TotalPopulation',
 'BirthRate']

In [10]:
in3.show(5)

+-----+----+--------+----------+---------+------+---------------+---------+
|Notes|Year|YearCode|     State|StateCode|Births|TotalPopulation|BirthRate|
+-----+----+--------+----------+---------+------+---------------+---------+
| null|2009|    2009|   Alabama|       01| 62475|        4757938|    13.13|
| null|2009|    2009|    Alaska|       02| 11324|         698895|    16.20|
| null|2009|    2009|   Arizona|       04| 92798|        6343154|    14.63|
| null|2009|    2009|  Arkansas|       05| 39808|        2896843|    13.74|
| null|2009|    2009|California|       06|527020|       36961229|    14.26|
+-----+----+--------+----------+---------+------+---------------+---------+
only showing top 5 rows



In [11]:
in3.registerTempTable('birth')

In [12]:
#Calculate the data for one or more cars


In [13]:
q1=sqlContext.sql("SELECT *, 100-total_est as 1orMore from zero ")

In [14]:
q1.show()

+---------+-------+----+---------+---------+-----------------+
|    geoid|   name|year|total_est| total_se|          1orMore|
+---------+-------+----+---------+---------+-----------------+
|04000US01|Alabama|2009|6.3250961|.16840608|       93.6749039|
|04000US01|Alabama|2010|6.5895853|.15879601|       93.4104147|
|04000US01|Alabama|2011|6.3611312| .1781458|       93.6388688|
|04000US01|Alabama|2012| 6.688168|   .15747|        93.311832|
|04000US01|Alabama|2013|6.5363503|.16953626|       93.4636497|
|04000US01|Alabama|2014|6.5472999|.16817892|       93.4527001|
|04000US01|Alabama|2015|6.5908065|.16147067|       93.4091935|
|04000US01|Alabama|2016|6.2311945|.16993542|       93.7688055|
|04000US01|Alabama|2017|5.9334354|.16884352|       94.0665646|
|04000US02| Alaska|2009| 9.685668|.45384088|90.31433200000001|
|04000US02| Alaska|2010|10.403362|  .537247|        89.596638|
|04000US02| Alaska|2011|9.6572495|.44409087|       90.3427505|
|04000US02| Alaska|2012|9.5159569|.38176191|       90.4

In [15]:
q1.registerTempTable("0and1more")

In [16]:
#Join all the car ownership together
q2=sqlContext.sql('''
select a.name,a.year,a.total_est as zero,b.total_est as one,a.1orMore, c.total_est as twoorMore,d.total_est as threeorMore,
e.total_est as fourorMore
FROM 0and1more a
JOIN One b on (a.name=b.name and a.year=b.year) 
JOIN Two c on (a.name=c.name and a.year=c.year)
JOIN Three d on (a.name=d.name and a.year=d.year) 
JOIN Four e  on (a.name=e.name and a.year=e.year) 
''')

In [17]:
q2.show(5)

+-------+----+---------+---------+----------+---------+-----------+----------+
|   name|year|     zero|      one|   1orMore|twoorMore|threeorMore|fourorMore|
+-------+----+---------+---------+----------+---------+-----------+----------+
|Alabama|2009|6.3250961|32.935886|93.6749039|60.739017|  23.794041| 7.3752837|
|Alabama|2010|6.5895853|32.708939|93.4104147|60.701473|  22.899515| 7.0476742|
|Alabama|2011|6.3611312|32.870529|93.6388688|60.768341|   23.09083| 7.1225657|
|Alabama|2012| 6.688168| 33.35326| 93.311832|59.958572|  23.136471| 7.3047509|
|Alabama|2013|6.5363503|33.146679|93.4636497|60.316967|  23.454721|  7.159307|
+-------+----+---------+---------+----------+---------+-----------+----------+
only showing top 5 rows



In [18]:
q2.registerTempTable('car')

In [19]:
#Join car and birth rate
q3=sqlContext.sql('''select c.*,b.BirthRate
from car c
join birth b on (c.name=b.State and c.year=b.Year)''')

In [20]:
q3.show(5)

+-------+----+---------+---------+----------+---------+-----------+----------+---------+
|   name|year|     zero|      one|   1orMore|twoorMore|threeorMore|fourorMore|BirthRate|
+-------+----+---------+---------+----------+---------+-----------+----------+---------+
|Alabama|2009|6.3250961|32.935886|93.6749039|60.739017|  23.794041| 7.3752837|    13.13|
|Alabama|2010|6.5895853|32.708939|93.4104147|60.701473|  22.899515| 7.0476742|    12.56|
|Alabama|2011|6.3611312|32.870529|93.6388688|60.768341|   23.09083| 7.1225657|    12.36|
|Alabama|2012| 6.688168| 33.35326| 93.311832|59.958572|  23.136471| 7.3047509|    12.12|
|Alabama|2013|6.5363503|33.146679|93.4636497|60.316967|  23.454721|  7.159307|    12.03|
+-------+----+---------+---------+----------+---------+-----------+----------+---------+
only showing top 5 rows



In [21]:
q3.registerTempTable('carbirth')

In [22]:
#Calculate the average data of carbirth 
q4=sqlContext.sql('''select name,
round(avg(BirthRate),2) as avgBirthRate,
round(avg(zero),2) as avgzero,
round(avg(one),2) as avgone,
round(avg(1orMore),2) as avg1orMore,
round(avg(twoorMore),2) as avgtwoorMore,
round(avg(threeorMore),2) as avgthreeorMore,
round(avg(fourorMore),2) as avgfourorMore


from carbirth group by  name
order by avgBirthRate desc''')

In [23]:
q4.show(5)

+--------------------+------------+-------+------+----------+------------+--------------+-------------+
|                name|avgBirthRate|avgzero|avgone|avg1orMore|avgtwoorMore|avgthreeorMore|avgfourorMore|
+--------------------+------------+-------+------+----------+------------+--------------+-------------+
|                Utah|       17.67|   4.38|  25.6|     95.62|       70.02|         28.36|         9.79|
|              Alaska|       15.45|   9.71| 31.02|     90.29|       59.27|         21.51|         6.83|
|               Texas|       14.77|   5.76| 34.13|     94.24|       60.11|         19.87|         5.56|
|District of Columbia|       14.63|  36.32| 44.21|     63.68|       19.47|          3.73|         0.88|
|        North Dakota|        14.4|   5.18| 29.08|     94.82|       65.73|         27.61|         10.3|
+--------------------+------------+-------+------+----------+------------+--------------+-------------+
only showing top 5 rows



In [25]:
q4.registerTempTable("avg")
#savefiles
q3.rdd.map(lambda i:",".join(str(j) for j in i)).saveAsTextFile('./carbirth.csv')
q4.rdd.map(lambda i:",".join(str(j) for j in i)).saveAsTextFile('./avgcarbirth.csv')